In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

pd.options.display.max_rows = 100
pd.options.display.max_columns = 100

In [2]:
import sys
sys.path.append("../")

from carbonify import BaseCarbone

# Exploring Base Carbone
- [Base Carbone](https://data.ademe.fr/datasets/base-carbone(r)) by ADEME 
- [Documentation](https://www.bilans-ges.ademe.fr/fr/accueil/contenu/index/page/presentation/siGras/0)

In [3]:
PATH = "../data/raw/base_carbone.csv"

baca = BaseCarbone(PATH)

## Visualization helper

In [4]:
import plotly.express as px

In [32]:
r,fig = baca.search("mangue",kind = "treemap")
fig

## A few comparisons

### Which emits more between planes and train?

In [18]:
id_train = 28145
id_avion = 28130

In [19]:
baca.search_by_id(id_train)

{'full_name': 'TGV 2019', 'emissions': 0.00173, 'unit': 'kgCO2e/passager.km'}

In [20]:
baca.search_by_id(id_avion)

{'full_name': 'Avion passagers Court courrier, 2018 AVEC trainées',
 'emissions': 0.258,
 'unit': 'kgCO2e/peq.km'}

In [21]:
baca.compare(id_avion,id_train,raise_unit_error=False)

Warning - First element unit is kgCO2e/peq.km and second one is kgCO2e/passager.km


149.13294797687863

In [4]:
train = baca.search("train")
avion = baca.search("avion")

For planes or TGVs already several questions arise about what to choose as hypothesis (trainées, distance, capacity)

## How much do I emit when I travel from NYC to Paris ?

About trainées https://www.carbone4.com/trainees-de-condensation-impact-climat

In [31]:
baca.evaluate_transportation_by_plane(5596,round_trip = True)

1701.184

## What's worse between a mango in France and a mango bought from South America?

In [33]:
r,fig = baca.search("mangue",kind = "treemap")
fig

In [35]:
baca.compare(33054,28425,raise_unit_error=False)

Warning - First element unit is kgCO2e/kg de poids net and second one is kgCO2e/kg


122.13114754098362

In [36]:
baca.compare(33055,28425,raise_unit_error=False)

Warning - First element unit is kgCO2e/kg de poids net and second one is kgCO2e/kg


5.368852459016394

Can we track back the factor 100 for emissions due to importations?

Let's say we import from Bogota by plane, the super easy LCA is : 
- Producing on site -> let's say it's similar than in France
- Transporting from agricultural site to France
- We don't consider transportation from Paris to your retailer

In [45]:
distance_bogota_paris = 8617

In [50]:
# Assuming 1kg of mangos (because cargos are in tons)
baca.evaluate_transportation_by_plane(distance_bogota_paris,round_trip=False,cargo = True,condensation_trails = False) * 0.001

16.3723

We find around the same numbers than in the Base Carbone 

In [57]:
easy_lca = (
pd.DataFrame([
    {"category":"production","name":"Production in Bogota","comment":"","value":0.12},
    {"category":"transportation","name":"Bogota to Paris","comment":"cargo plane","value":16.3723},
    {"category":"transportation","name":"Paris to retailer","comment":"unregarded","value":0},
]).assign(percent = lambda x : x["value"] / x["value"].sum())
.style.background_gradient(cmap = "Blues",subset = "percent")
)

easy_lca

,category,name,comment,value,percent
0,production,Production in Bogota,,0.120000,0.007276
1,transportation,Bogota to Paris,cargo plane,16.372300,0.992724
2,transportation,Paris to retailer,unregarded,0.000000,0.000000
